Cấu hình và phụ thuộc

Mục đích: thiết lập tham số huấn luyện và thư viện cần dùng.

In [8]:
import tensorflow as tf
print("TensorFlow:", tf.__version__)

from transformers import AutoTokenizer, TFAutoModel
MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"

tok = AutoTokenizer.from_pretrained(MODEL_NAME)
bert = TFAutoModel.from_pretrained(MODEL_NAME, from_pt=True)

print("Tokenizer OK:", type(tok).__name__)
print("Model OK:", type(bert).__name__)


TensorFlow: 2.16.2


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Tokenizer OK: BertTokenizerFast
Model OK: TFBertModel


BƯỚC 1 — Đưa đầu vào (Input)

Mục tiêu

Đọc dữ liệu đã tiền xử lý (examples.parquet, vocab_meta.json).

Biến văn bản text thành token ID và attention mask mà BioBERT hiểu (tokenizer của BioBERT).

Tạo tf.data.Dataset cho train và validation (chỉ chuẩn bị đầu vào, chưa xây mô hình).

In [9]:
# =========================
# BƯỚC 1 — ĐƯA ĐẦU VÀO (INPUT)
# =========================

# 1) Import các thư viện cần thiết
import json, random
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import GroupShuffleSplit
from transformers import AutoTokenizer

# 2) Cấu hình cơ bản cho việc tạo batch
MAX_LENGTH = 256   # độ dài tối đa của chuỗi token (cắt nếu dài hơn, pad nếu ngắn hơn)
BATCH_SIZE = 8     # kích thước lô (batch)
VAL_RATIO  = 0.15  # tỉ lệ dành cho validation
SEED = 42          # hạt giống ngẫu nhiên để tái lập kết quả

# 3) Cố định đường dẫn: notebook đang ở 'notebooks/', còn dữ liệu ở '../data/proc/'
DATA_ROOT = Path("..") / "data" / "proc"
PARQUET_PATH = DATA_ROOT / "examples.parquet"
VOCAB_PATH   = DATA_ROOT / "vocab_meta.json"

print("Đang dùng:")
print(" - PARQUET_PATH:", PARQUET_PATH.resolve())
print(" - VOCAB_PATH  :", VOCAB_PATH.resolve())

# 4) Kiểm tra sự tồn tại của file dữ liệu
if not PARQUET_PATH.exists():
    raise FileNotFoundError(
        f"Không tìm thấy {PARQUET_PATH}. Hãy chạy Notebook tiền xử lý để sinh file, "
        f"hoặc chỉnh lại đường dẫn cho đúng vị trí thực tế."
    )
if not VOCAB_PATH.exists():
    raise FileNotFoundError(
        f"Không tìm thấy {VOCAB_PATH}. Hãy kiểm tra Notebook tiền xử lý hoặc đường dẫn."
    )

# 5) Kiểm soát ngẫu nhiên để kết quả ổn định
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 6) Đọc dữ liệu bảng đã tiền xử lý và metadata nhãn
df = pd.read_parquet(PARQUET_PATH)
with open(VOCAB_PATH, "r") as f:
    meta = json.load(f)

icd_vocab        = meta["icd_vocab"]              # danh sách ICD-block (vocab)
lab_vocab_items  = meta["lab_vocab_items"]        # danh sách itemid xét nghiệm (vocab)
itemid_to_label  = {int(k): v for k, v in meta["itemid_to_label"].items()}

n_icd = len(icd_vocab)
n_lab = len(lab_vocab_items)

print(f"Số mẫu: {len(df)} | n_icd: {n_icd} | n_lab: {n_lab}")

# 7) Đảm bảo 2 cột nhãn là mảng float32 để dùng với BinaryCrossentropy (ở các bước sau)
df["y_icd"] = df["y_icd"].apply(lambda a: np.asarray(a, dtype=np.float32))
df["y_lab"] = df["y_lab"].apply(lambda a: np.asarray(a, dtype=np.float32))

# 8) Chia train/validation theo subject_id (tránh cùng 1 bệnh nhân xuất hiện ở cả 2 tập)
groups = df["subject_id"].values
gss = GroupShuffleSplit(n_splits=1, test_size=VAL_RATIO, random_state=SEED)
train_idx, val_idx = next(gss.split(df, groups=groups))
train_df = df.iloc[train_idx].reset_index(drop=True)
val_df   = df.iloc[val_idx].reset_index(drop=True)

print("Số mẫu train:", len(train_df))
print("Số mẫu val  :", len(val_df))

# 9) Chuẩn bị tokenizer của BioBERT
#    - Nếu bạn đã chạy ô trước và có biến 'tok', ta tận dụng lại.
#    - Nếu chưa, tự nạp từ MODEL_NAME.
try:
    tokenizer = tok  # tái dùng tokenizer đã nạp
except NameError:
    MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 10) Hàm mã hóa 1 lô văn bản sang token ID & attention mask (đầu vào mô hình)
def encode_text_batch(text_list, max_length):
    """
    Biến danh sách chuỗi text thành dict numpy arrays:
      - input_ids: ID của token theo từ điển của BioBERT
      - attention_mask: 1 ở vị trí có token thực, 0 ở chỗ padding
    Tất cả được pad/cắt về đúng max_length.
    """
    enc = tokenizer(
        list(text_list),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors=None   # trả về list/np array thay vì tensor của HF
    )
    # Ép kiểu sang int32 để tiện đưa vào tf.data
    enc["input_ids"] = np.asarray(enc["input_ids"], dtype=np.int32)
    enc["attention_mask"] = np.asarray(enc["attention_mask"], dtype=np.int32)
    return enc

# 11) Tạo tf.data.Dataset theo kiểu generator để tiết kiệm RAM và linh hoạt chiều dài chuỗi
def make_tfds_from_df(frame, batch_size, max_length):
    """
    Từ DataFrame gồm cột 'text', 'y_icd', 'y_lab' tạo Dataset:
      - X = {"input_ids": [B, L], "attention_mask": [B, L]}
      - y = {"icd": [B, n_icd],   "lab": [B, n_lab]}
    """
    texts = frame["text"].fillna("").tolist()
    y_icd = np.stack(frame["y_icd"].to_list()).astype(np.float32)
    y_lab = np.stack(frame["y_lab"].to_list()).astype(np.float32)

    def gen():
        """Phát từng phần tử (x_i, y_i) để tf.data có thể làm padded_batch sau đó."""
        # Mã hóa theo từng block nhỏ bằng chính 'batch_size' để giảm thời gian tokenize
        N = len(texts)
        start = 0
        while start < N:
            end = min(start + batch_size, N)
            enc = encode_text_batch(texts[start:end], max_length=max_length)
            for i in range(end - start):
                yield (
                    {
                        "input_ids": enc["input_ids"][i],
                        "attention_mask": enc["attention_mask"][i],
                    },
                    {
                        "icd": y_icd[start + i],
                        "lab": y_lab[start + i],
                    }
                )
            start = end

    # Khai báo "hình thù" của từng phần tử để tf.data biết cách đóng gói/padding
    output_signature = (
        {
            "input_ids": tf.TensorSpec(shape=(None,), dtype=tf.int32),
            "attention_mask": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        },
        {
            "icd": tf.TensorSpec(shape=(n_icd,), dtype=tf.float32),
            "lab": tf.TensorSpec(shape=(n_lab,), dtype=tf.float32),
        },
    )

    ds = tf.data.Dataset.from_generator(gen, output_signature=output_signature)

    # Đóng gói thành batch và pad/truncate về MAX_LENGTH cho đồng nhất
    ds = ds.padded_batch(
        batch_size,
        padded_shapes=(
            {"input_ids": [MAX_LENGTH], "attention_mask": [MAX_LENGTH]},
            {"icd": [n_icd], "lab": [n_lab]},
        ),
        padding_values=(
            {"input_ids": tf.constant(0, tf.int32), "attention_mask": tf.constant(0, tf.int32)},
            {"icd": tf.constant(0.0, tf.float32),   "lab": tf.constant(0.0, tf.float32)},
        ),
        drop_remainder=False
    )
    # Tối ưu hiệu năng đọc
    return ds.prefetch(tf.data.AUTOTUNE)

# 12) Tạo train_ds và val_ds để dùng ở Bước 2 (BioBERT) và Bước 3 (Heads)
train_ds = make_tfds_from_df(train_df, BATCH_SIZE, MAX_LENGTH).shuffle(1024)
val_ds   = make_tfds_from_df(val_df,   BATCH_SIZE, MAX_LENGTH)

# 13) Kiểm tra nhanh một batch để hiểu đúng ý
for batch in train_ds.take(1):
    X, Y = batch
    print("Kích thước đầu vào (X):", {k: v.shape for k, v in X.items()})
    print("Kích thước nhãn (Y):   ", {k: v.shape for k, v in Y.items()})
    # In thử 120 ký tự đầu của văn bản đầu tiên trong batch (để thấy dữ liệu gốc)
    # Lưu ý: X không chứa 'text' thô, chỉ có token; ta lấy lại từ train_df để minh họa
    print("Ví dụ text gốc (rút gọn):", train_df.iloc[0]["text"][:120].replace("\n", " "))
    break


Đang dùng:
 - PARQUET_PATH: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc/examples.parquet
 - VOCAB_PATH  : /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc/vocab_meta.json
Số mẫu: 828 | n_icd: 200 | n_lab: 50
Số mẫu train: 717
Số mẫu val  : 111
Kích thước đầu vào (X): {'input_ids': TensorShape([8, 256]), 'attention_mask': TensorShape([8, 256])}
Kích thước nhãn (Y):    {'icd': TensorShape([8, 200]), 'lab': TensorShape([8, 50])}
Ví dụ text gốc (rút gọn): EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: ___ year-old female with cirrhosis, jaundice. TECHNIQUE:


BƯỚC 2 — Xử lý trong BioBERT (tạo vector ngữ nghĩa)

Mục tiêu

Đưa batch input_ids + attention_mask vào BioBERT.

Lấy ra vector biểu diễn câu (dùng token đặc biệt [CLS]).

Kiểm tra kích thước và giá trị điển hình để xác nhận “dịch văn bản → ngôn ngữ số”.

In [10]:
# =========================
# BƯỚC 2 — XỬ LÝ TRONG BIOBERT
# =========================

import tensorflow as tf
from transformers import TFAutoModel

MODEL_NAME = "dmis-lab/biobert-base-cased-v1.1"
DROPOUT = 0.1  # dropout nhẹ để ổn định biểu diễn khi train (ở bước này ta chỉ forward thử)

# 2.1 Nạp backbone BioBERT (TF). Nếu đã có biến `bert` từ trước thì tái dùng.
try:
    _ = bert  # kiểm tra biến đã tồn tại
except NameError:
    # from_pt=True: nhiều checkpoint BioBERT là bản PyTorch; cờ này cho phép nạp sang TF
    bert = TFAutoModel.from_pretrained(MODEL_NAME, from_pt=True)

# 2.2 Định nghĩa inputs chuẩn Keras khớp với batch của tf.data
input_ids      = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

# 2.3 Cho dữ liệu đi qua BioBERT để lấy "dấu vân tay" [CLS]
#     - last_hidden_state: [batch, seq_len, hidden_size]
#     - lấy vị trí [:, 0, :] tương ứng token [CLS] (đại diện toàn câu)
bert_outputs = bert(input_ids, attention_mask=attention_mask, training=False)
cls_vec = bert_outputs.last_hidden_state[:, 0, :]  # [batch, hidden_size], ví dụ [8, 768]

# (tùy chọn) thêm dropout nhẹ để quen với train; đây không thay đổi kích thước
cls_vec = tf.keras.layers.Dropout(DROPOUT, name="cls_dropout")(cls_vec)

# 2.4 Gói thành một "encoder model" xuất ra vector CLS — dùng lại ở Bước 3
encoder = tf.keras.Model(
    inputs={"input_ids": input_ids, "attention_mask": attention_mask},
    outputs={"cls": cls_vec},
    name="biobert_encoder_cls"
)

# 2.5 Chạy thử 1 batch để kiểm tra kích thước và vài số đầu tiên
for X_batch, Y_batch in train_ds.take(1):
    out = encoder(X_batch, training=False)   # forward pass
    cls_batch = out["cls"].numpy()           # [BATCH_SIZE, HIDDEN]
    print("Kích thước vector CLS:", cls_batch.shape)
    # In 8 giá trị đầu của 1 mẫu để thấy "ngôn ngữ số" (chỉ minh họa)
    print("8 số đầu của CLS[0]:", np.round(cls_batch[0][:8], 4).tolist())
    break


Kích thước vector CLS: (8, 768)
8 số đầu của CLS[0]: [0.13459999859333038, -0.023900000378489494, -0.17829999327659607, -0.04659999907016754, -0.436599999666214, -0.028200000524520874, 0.3319000005722046, 0.2379000037908554]


Ý nghĩa của kết quả

Bạn sẽ nhận được ma trận kích thước [BATCH_SIZE, HIDDEN] (thường HIDDEN = 768).

Mỗi hàng là “dấu vân tay số học” của 1 văn bản — chứa ngữ cảnh y khoa đã được BioBERT “đọc hiểu”.

Đây chính là đầu vào cho “phần ra quyết định” ở Bước 3.

BƯỚC 3 — Classifier head (Phần ra quyết định)

Mục tiêu

Gắn 2 “đầu ra quyết định” (Dense) lên vector CLS từ BioBERT:

Đầu ICD: dự đoán đa nhãn các ICD-block.

Đầu Lab: dự đoán đa nhãn các xét nghiệm sớm.

Mỗi đầu trả về logits (số thực âm/dương). Khi qua sigmoid sẽ thành xác suất 0–1 cho từng nhãn.

Kiểm tra kích thước đầu ra và xem thử top-k dự đoán trên một batch (chưa huấn luyện, chỉ để minh họa dòng chảy dữ liệu).

In [11]:
# =========================
# BƯỚC 3 — CLASSIFIER HEAD
# =========================

import tensorflow as tf
import numpy as np

# 3.1 Lấy đầu ra CLS từ encoder (đã có ở Bước 2)
#     encoder.inputs:  {"input_ids", "attention_mask"}
#     encoder.outputs: {"cls"}  (vector kích thước [batch, hidden_size])
cls_output = encoder.outputs[0]  # tương đương encoder.get_layer("cls_dropout").output

# 3.2 Thêm hai lớp Dense (tuyến tính) cho 2 tác vụ đa nhãn
#     - Không dùng activation tại đây để mô hình trả về "logits".
#     - Logits giúp ta dùng BinaryCrossentropy(from_logits=True) ở bước tính loss.
icd_logits = tf.keras.layers.Dense(n_icd, name="icd_logits")(cls_output)  # [batch, n_icd]
lab_logits = tf.keras.layers.Dense(n_lab, name="lab_logits")(cls_output)  # [batch, n_lab]

# 3.3 Tạo mô hình Keras đầy đủ (encoder + 2 heads)
multitask_model = tf.keras.Model(
    inputs=encoder.inputs,                    # {"input_ids", "attention_mask"}
    outputs={"icd": icd_logits, "lab": lab_logits},  # hai nhánh logits
    name="biobert_multitask_heads"
)

# 3.4 Kiểm tra kiến trúc
multitask_model.summary(line_length=120)

# 3.5 Chạy thử một batch để xem kích thước logits và minh họa xác suất (chưa huấn luyện)
for X_batch, Y_batch in train_ds.take(1):
    # logits thô từ hai đầu
    logits = multitask_model.predict(X_batch, verbose=0)
    icd_log = logits["icd"]   # [BATCH_SIZE, n_icd]
    lab_log = logits["lab"]   # [BATCH_SIZE, n_lab]
    print("Kích thước icd_logits:", icd_log.shape)
    print("Kích thước lab_logits:", lab_log.shape)

    # Chuyển logits -> xác suất bằng sigmoid để "đọc" dễ hơn
    probs_icd = tf.sigmoid(icd_log).numpy()   # [BATCH_SIZE, n_icd]
    probs_lab = tf.sigmoid(lab_log).numpy()   # [BATCH_SIZE, n_lab]

    # Lấy top-k (ví dụ k=5) cho mẫu đầu tiên của batch, để minh họa "ô xác suất"
    k = 5
    top_icd_idx = probs_icd[0].argsort()[-k:][::-1]  # chỉ số 5 nhãn ICD có xác suất cao nhất
    top_lab_idx = probs_lab[0].argsort()[-k:][::-1]  # chỉ số 5 xét nghiệm có xác suất cao nhất

    # In ra chỉ số và xác suất tương ứng
    print("Top-5 ICD indices:", top_icd_idx.tolist())
    print("Top-5 ICD probs  :", np.round(probs_icd[0][top_icd_idx], 4).tolist())

    print("Top-5 Lab indices:", top_lab_idx.tolist())
    print("Top-5 Lab probs  :", np.round(probs_lab[0][top_lab_idx], 4).tolist())

    # (Tuỳ chọn) Map chỉ số -> mã ICD-block / itemid xét nghiệm để dễ đọc
    top_icd_codes = [icd_vocab[i] for i in top_icd_idx]
    top_lab_itemids = [lab_vocab_items[j] for j in top_lab_idx]
    print("Top-5 ICD codes:", top_icd_codes)
    print("Top-5 Lab itemids:", top_lab_itemids)
    break


Model: "biobert_multitask_heads"
________________________________________________________________________________________________________________________
 Layer (type)                       Output Shape                        Param #     Connected to                        
 input_ids (InputLayer)             [(None, None)]                      0           []                                  
                                                                                                                        
 attention_mask (InputLayer)        [(None, None)]                      0           []                                  
                                                                                                                        
 tf_bert_model_1 (TFBertModel)      TFBaseModelOutputWithPoolingAndCr   108310272   ['input_ids[0][0]',                 
                                    ossAttentions(last_hidden_state=(                'attention_mask[0][0]']            

Ý nghĩa của kết quả

Bạn sẽ thấy hai ma trận: icd_logits có kích thước [BATCH_SIZE, n_icd], lab_logits có kích thước [BATCH_SIZE, n_lab].

Sau khi áp sigmoid, bạn sẽ nhận được hai ma trận xác suất probs_icd, probs_lab cùng kích thước, mỗi cột là “một ô” xác suất mà bạn mô tả (ví dụ 0.85 cho I21, 0.92 cho Troponin…).

Đây chính là “phần ra quyết định” trước khi so sánh với nhãn thật ở Bước 4.

BƯỚC 4 — So sánh với nhãn thật (Ground truth)

Mục tiêu

Lấy dự đoán từ Bước 3 (logits → xác suất).

So sánh với nhãn thật y_icd, y_lab cho một mẫu trong validation:

Xem những nhãn nào mô hình dự đoán (theo ngưỡng 0.5) so với nhãn thật.

Xem Top-k (ví dụ k=5) nhãn có xác suất cao nhất và đối chiếu nhãn thật.

In [12]:
# ===============================
# BƯỚC 4 — SO SÁNH VỚI NHÃN THẬT
# ===============================

import numpy as np
import tensorflow as tf

# 1) Lấy một batch từ validation để so sánh
X_batch, Y_batch = next(iter(val_ds))  # X_batch: dict input_ids/attention_mask; Y_batch: dict icd/lab

# 2) Chạy mô hình đa nhiệm (từ Bước 3) để lấy logits
logits = multitask_model.predict(X_batch, verbose=0)
icd_log = logits["icd"]   # [BATCH_SIZE, n_icd]
lab_log = logits["lab"]   # [BATCH_SIZE, n_lab]

# 3) Đổi logits -> xác suất (0..1) bằng sigmoid để dễ "đọc"
probs_icd = tf.sigmoid(icd_log).numpy()
probs_lab = tf.sigmoid(lab_log).numpy()

# 4) Chọn một mẫu trong batch để nhìn chi tiết (ví dụ mẫu đầu tiên)
idx = 0
y_icd_true = Y_batch["icd"][idx].numpy()   # vector multi-hot thật [n_icd]
y_lab_true = Y_batch["lab"][idx].numpy()   # vector multi-hot thật [n_lab]

# 5) Chiến lược A: So sánh theo NGƯỠNG 0.5 (mặc định)
THR_ICD = 0.5
THR_LAB = 0.5

y_icd_pred_bin = (probs_icd[idx] >= THR_ICD).astype(int)  # 1 nếu xác suất ≥ 0.5
y_lab_pred_bin = (probs_lab[idx] >= THR_LAB).astype(int)

true_icd_idx = np.where(y_icd_true == 1)[0]               # các chỉ số nhãn ICD thật sự có
pred_icd_idx = np.where(y_icd_pred_bin == 1)[0]           # các chỉ số nhãn ICD mô hình chọn theo ngưỡng
hit_icd_idx  = np.intersect1d(true_icd_idx, pred_icd_idx) # giao nhau = dự đoán đúng

true_lab_idx = np.where(y_lab_true == 1)[0]
pred_lab_idx = np.where(y_lab_pred_bin == 1)[0]
hit_lab_idx  = np.intersect1d(true_lab_idx, pred_lab_idx)

print("=== So sánh theo NGƯỠNG 0.5 (mẫu idx=0) ===")
print(f"ICD: thật={len(true_icd_idx)} | dự đoán={len(pred_icd_idx)} | đúng={len(hit_icd_idx)}")
print(f"LAB: thật={len(true_lab_idx)} | dự đoán={len(pred_lab_idx)} | đúng={len(hit_lab_idx)}")

# 6) Chiến lược B: So sánh theo TOP-K (ví dụ k=5)
K_ICD = 5
K_LAB = 5

top_icd_idx = probs_icd[idx].argsort()[-K_ICD:][::-1]  # chỉ số K nhãn ICD có xác suất cao nhất
top_lab_idx = probs_lab[idx].argsort()[-K_LAB:][::-1]

print("\n=== So sánh theo TOP-K ===")
print("Top-5 ICD (index → mã → xác suất → có phải nhãn thật?):")
for j in top_icd_idx:
    code = icd_vocab[j]
    p = float(probs_icd[idx][j])
    is_true = int(y_icd_true[j])  # 1 nếu là nhãn thật
    print(f"  idx={j:4d} | ICD={code:5s} | p={p:0.3f} | nhãn_thật={is_true}")

print("\nTop-5 Lab (index → itemid → tên → xác suất → có phải nhãn thật?):")
for j in top_lab_idx:
    itemid = int(lab_vocab_items[j])
    name = itemid_to_label.get(itemid, str(itemid))
    p = float(probs_lab[idx][j])
    is_true = int(y_lab_true[j])
    print(f"  idx={j:4d} | itemid={itemid} | {name} | p={p:0.3f} | nhãn_thật={is_true}")

# 7) (Tùy chọn) In danh sách nhãn đúng theo ngưỡng để hình dung
def map_icd_indices(indices):
    return [icd_vocab[i] for i in indices]

def map_lab_indices(indices):
    return [(int(lab_vocab_items[i]), itemid_to_label.get(int(lab_vocab_items[i]), str(lab_vocab_items[i]))) for i in indices]

print("\nICD — nhãn_thật (mã):", map_icd_indices(true_icd_idx)[:20], "...")  # cắt bớt nếu dài
print("ICD — dự_đoán_theo_ngưỡng (mã):", map_icd_indices(pred_icd_idx)[:20], "...")
print("ICD — giao_nhau_đúng (mã):", map_icd_indices(hit_icd_idx)[:20], "...")

print("\nLAB — nhãn_thật (itemid, tên):", map_lab_indices(true_lab_idx)[:10], "...")
print("LAB — dự_đoán_theo_ngưỡng (itemid, tên):", map_lab_indices(pred_lab_idx)[:10], "...")
print("LAB — giao_nhau_đúng (itemid, tên):", map_lab_indices(hit_lab_idx)[:10], "...")


=== So sánh theo NGƯỠNG 0.5 (mẫu idx=0) ===
ICD: thật=0 | dự đoán=90 | đúng=0
LAB: thật=0 | dự đoán=27 | đúng=0

=== So sánh theo TOP-K ===
Top-5 ICD (index → mã → xác suất → có phải nhãn thật?):
  idx=  53 | ICD=790   | p=0.796 | nhãn_thật=0
  idx=  72 | ICD=278   | p=0.782 | nhãn_thật=0
  idx=  14 | ICD=I48   | p=0.780 | nhãn_thật=0
  idx=   9 | ICD=Y92   | p=0.779 | nhãn_thật=0
  idx=  79 | ICD=412   | p=0.764 | nhãn_thật=0

Top-5 Lab (index → itemid → tên → xác suất → có phải nhãn thật?):
  idx=  11 | itemid=50902 | Chloride | p=0.844 | nhãn_thật=0
  idx=  44 | itemid=51274 | PT | p=0.807 | nhãn_thật=0
  idx=  37 | itemid=51116 | Lymphocytes | p=0.798 | nhãn_thật=0
  idx=  28 | itemid=51498 | Specific Gravity | p=0.688 | nhãn_thật=0
  idx=   7 | itemid=51279 | Red Blood Cells | p=0.683 | nhãn_thật=0

ICD — nhãn_thật (mã): [] ...
ICD — dự_đoán_theo_ngưỡng (mã): ['401', 'Z87', 'E87', 'Y92', 'V15', 'K21', 'I48', 'V45', 'N17', 'G47', 'Z86', '530', '427', 'N18', 'Z20', 'V10', 'Z85', 'F1

Ý nghĩa của kết quả

Bạn thấy rõ cách đối chiếu:

“Dự đoán theo ngưỡng” (≥ 0.5) vs. nhãn thật → đếm đúng/sai.

“Dự đoán Top-k” → xem 5 nhãn cao nhất và đánh dấu nhãn nào trùng nhãn thật.

Đây là bước không tính loss (chưa huấn luyện), chỉ so sánh để hiểu dòng chảy dự đoán ↔ nhãn.
(Loss sẽ thiết lập ở Bước 5.)

BƯỚC 5 — Tính sai số (Loss)

Mục tiêu

Khai báo hàm mất mát cho 2 nhánh đầu ra (ICD và Lab) đúng với thiết kế đa nhãn.

Biên dịch (compile) mô hình với optimizer/metrics, chưa huấn luyện.

Tính thử loss & AUPRC trên 1 batch để thấy mô hình đang ở mức “ngẫu nhiên” trước khi học.

In [13]:
# ======================================
# BƯỚC 5 — TÍNH SAI SỐ (LOSS) & COMPILE
# ======================================

import tensorflow as tf

# 1) Định nghĩa loss cho BÀI TOÁN ĐA NHÃN:
#    - BinaryCrossentropy(from_logits=True): phù hợp khi đầu ra là "logits" (chưa qua sigmoid)
#    - Mỗi nhánh có một loss riêng, Keras sẽ cộng có trọng số thành 'loss' tổng.
losses = {
    "icd": tf.keras.losses.BinaryCrossentropy(from_logits=True),
    "lab": tf.keras.losses.BinaryCrossentropy(from_logits=True),
}

# 2) Trọng số cho từng nhánh (có thể chỉnh nếu muốn ưu tiên 1 nhánh nào đó)
loss_weights = {"icd": 1.0, "lab": 1.0}

# 3) Chỉ số đánh giá: AUPRC (Area Under Precision–Recall) dạng multi-label
#    - Cho biết chất lượng mô hình ở bối cảnh mất cân bằng nhãn.
metrics = {
    "icd": [tf.keras.metrics.AUC(curve="PR", multi_label=True, name="AUPRC")],
    "lab": [tf.keras.metrics.AUC(curve="PR", multi_label=True, name="AUPRC")],
}

# 4) Tối ưu hóa: Adam với learning rate nhỏ (fine-tune mô hình ngôn ngữ)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

# 🔒 Đóng băng phần backbone BioBERT (không huấn luyện phần này)
backbone = next(l for l in multitask_model.layers if "bert" in l.name)
backbone.trainable = False
print("Backbone BioBERT trainable:", backbone.trainable)

# 5) Biên dịch (compile) mô hình
multitask_model.compile(
    optimizer=optimizer,
    loss=losses,
    loss_weights=loss_weights,
    metrics=metrics
)

# 6) TÍNH THỬ LOSS TRƯỚC KHI HUẤN LUYỆN
#    - Lấy 1 batch từ validation để "đo" loss/AUPRC ban đầu
X_batch, Y_batch = next(iter(val_ds))

results = multitask_model.evaluate(
    X_batch, Y_batch,
    verbose=0,            # in gọn
    return_dict=True      # trả về dict để đọc tên trường cho rõ
)

print("Kết quả trên 1 batch (trước khi train):")
for k, v in results.items():
    # Làm tròn cho dễ nhìn
    try:
        print(f"  {k}: {v:.4f}")
    except Exception:
        print(f"  {k}: {v}")


Backbone BioBERT trainable: False
Kết quả trên 1 batch (trước khi train):
  loss: 1.4856
  icd_logits_loss: 0.7190
  lab_logits_loss: 0.7666
  icd_logits_AUPRC: 0.0261
  lab_logits_AUPRC: 0.0000


BƯỚC 6 — Cập nhật (Training)

Mục tiêu

Huấn luyện toàn bộ mô hình: BioBERT + 2 đầu ra (ICD, Lab) bằng Adam.

Dùng EarlyStopping để dừng sớm nếu không cải thiện.

Lưu checkpoint tốt nhất theo chỉ số thẩm định (validation).

In [14]:
# ==================================
# BƯỚC 6 — HUẤN LUYỆN (TRAINING)
# ==================================

import tensorflow as tf
from pathlib import Path

EPOCHS = 3                # số vòng lặp qua toàn bộ dữ liệu
OUT_DIR = Path("checkpoints_biobert_multitask")  # nơi lưu checkpoint tốt nhất
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) EarlyStopping: dừng sớm nếu không cải thiện AUPRC trên validation
#    - Bạn có thể chọn theo nhánh "icd" hoặc "lab". Ở đây lấy "icd" làm tiêu chí.
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_icd_logits_AUPRC",  # tên metric do Keras tự đặt: <output>_<metric>
    mode="max",
    patience=1,                      # nếu 1 epoch không cải thiện thì dừng
    restore_best_weights=True        # quay về trọng số tốt nhất
)

# 2) ModelCheckpoint: lưu checkpoint có AUPRC tốt nhất
ckpt_path = OUT_DIR / "best"
ckpt_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(ckpt_path),
    monitor="val_icd_logits_AUPRC",
    mode="max",
    save_best_only=True,
    save_weights_only=False,         # lưu cả cấu trúc + weights (SavedModel)
    verbose=1
)

# 3) Huấn luyện
history = multitask_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stop, ckpt_cb],
    verbose=1
)

# 4) Sau khi fit, evaluate lại trên validation để xem tổng quan
final_eval = multitask_model.evaluate(val_ds, return_dict=True, verbose=1)
print("\nĐánh giá cuối cùng trên validation:")
for k, v in final_eval.items():
    try:
        print(f"  {k}: {v:.4f}")
    except Exception:
        print(f"  {k}: {v}")

# 5) Lưu thêm một bản "final" (phòng khi muốn dùng ngay phiên bản hiện tại)
final_path = OUT_DIR / "final"
multitask_model.save(final_path)
print(f"\nĐã lưu mô hình hiện tại tại: {final_path.resolve()}")
print(f"Checkpoint tốt nhất (theo val_icd_logits_AUPRC) tại: {ckpt_path.resolve()}")


Epoch 1/3
     90/Unknown - 120s 1s/step - loss: 1.3712 - icd_logits_loss: 0.6643 - lab_logits_loss: 0.7069 - icd_logits_AUPRC: 0.0459 - lab_logits_AUPRC: 0.0076

2025-09-08 15:42:01.032424: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-09-08 15:42:18.976215: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 1: val_icd_logits_AUPRC improved from -inf to 0.03958, saving model to checkpoints_biobert_multitask/best
INFO:tensorflow:Assets written to: checkpoints_biobert_multitask/best/assets


INFO:tensorflow:Assets written to: checkpoints_biobert_multitask/best/assets


90/90 [==============================] - 171s 2s/step - loss: 1.3712 - icd_logits_loss: 0.6643 - lab_logits_loss: 0.7069 - icd_logits_AUPRC: 0.0459 - lab_logits_AUPRC: 0.0076 - val_loss: 1.2425 - val_icd_logits_loss: 0.6034 - val_lab_logits_loss: 0.6391 - val_icd_logits_AUPRC: 0.0396 - val_lab_logits_AUPRC: 0.0000e+00
Epoch 2/3
90/90 [==============================] - ETA: 0s - loss: 1.1564 - icd_logits_loss: 0.5652 - lab_logits_loss: 0.5911 - icd_logits_AUPRC: 0.0465 - lab_logits_AUPRC: 0.0053
Epoch 2: val_icd_logits_AUPRC did not improve from 0.03958
14/14 [==============================] - 18s 1s/step - loss: 1.2425 - icd_logits_loss: 0.6034 - lab_logits_loss: 0.6391 - icd_logits_AUPRC: 0.0396 - lab_logits_AUPRC: 0.0000e+00

Đánh giá cuối cùng trên validation:
  loss: 1.2425
  icd_logits_loss: 0.6034
  lab_logits_loss: 0.6391
  icd_logits_AUPRC: 0.0396
  lab_logits_AUPRC: 0.0000


2025-09-08 15:45:19.949599: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


INFO:tensorflow:Assets written to: checkpoints_biobert_multitask/final/assets


INFO:tensorflow:Assets written to: checkpoints_biobert_multitask/final/assets



Đã lưu mô hình hiện tại tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/notebooks/checkpoints_biobert_multitask/final
Checkpoint tốt nhất (theo val_icd_logits_AUPRC) tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/notebooks/checkpoints_biobert_multitask/best


Ý nghĩa của kết quả

loss, icd_loss, lab_loss trên validation sẽ giảm dần qua epoch; AUPRC sẽ tăng dần nếu dữ liệu/tiền xử lý ổn.

Sau khi huấn luyện, mô hình bắt đầu dự đoán hợp lý hơn (ít “tràn nhãn” ≥ 0.5, top-k sát thực tế hơn).